# Plot paired and unpaired synapses

This notebook assumes that all of the studies are orginized into a standard Python data structure. All of the plots are done using plot.ly, so you should look at the appropriate documentation for that if you want to change them.

The variable `study_list` constains a list of studies, where each study is represented as a pyton dictionary. The keys in the dictionary currently include the study name, the subject id, the type of study (aversion, control), and if the fish learned or not. 

The python command `list(study_list[0])` will return a list of the current values that are available for a study.

The study dictionary may also contain a set of calculated synapse pairings.  Each pair is represented as a python dictionary, whose key is the radius with which the pairing was determined, and whose value is a PANDA, with the synapse positions of the pair. Each PANDA has included in it a set of useful values from the study, including the study id, the type of study, the radius for the pair, etc.  Pairs are also transformed using the alignment points to put them into a common coordinate space.  The sets of pairs provided are:

* PairedBefore
* PairedAfter
* UnpairedBefore
* UnpairedAfter
* AlignedPairedBefore
* AlignedPairedAfter
* AlignedUnpairedBefore
* AlignedUnpairedAfter

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Load up all of the libraries we need

import synapse_plot_utils as sp
import synapse_utils
import synapse_plot_config as plot_config
from synapse_plot_config import plot_synapses, trace_color, position_layout, step_buttons, studytypes
from scipy.spatial import ConvexHull

import pandas as pd
import numpy as np
import xarray as xr
import plotly as py
import plotly.graph_objs as go
import matplotlib
import csv
import hdbscan

from matplotlib import cm

import colorlover as cl

# configure plot.ly to run in a notebook.
py.offline.init_notebook_mode(connected=True)

# Load Data

In [ ]:
studyid = '10DJ@2PS-H8ZM-3F2G'
studyset = synapse_utils.fetch_studies(studyid)
studylist = studyset['Studies']
study_types = studytypes(studylist)

<H1> Compute some basic Statistics

In [ ]:
# Set what radius/studies you want to have....

cols = ['ID', 'Type', 'Radius', 'Unpaired Before', 'Unpaired After','Paired',]
c1, c2, c3, c4, c5, c6 = [],[],[],[],[],[]

syn_pair_radii = list(studylist[0]['UnpairedBefore'])

for s in studylist:
    for r in syn_pair_radii:
        c1.append(s['Study'])
        c2.append(s['Type'])
        c3.append(r)
        c4.append(len(s['UnpairedBefore'][r]['Data'].index))
        c5.append(len(s['UnpairedAfter'][r]['Data'].index))
        c6.append(len(s['PairedBefore'][r]['Data'].index))
    
trace = go.Table(
    header=dict(values=cols,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[c1, c2, c3, c4, c5, c6],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

data = [trace] 
py.offline.iplot(data, filename = 'statistics_table')

<H1> Plot histograms of pair distance for all of the studies

In [ ]:
pair_distance = { k : pd.Series() for k in study_types}

rmax = max(studylist[0]['PairedAfter'])

for s in studylist:
    # Get the pairing for the maximum radius...
    before = s['PairedBefore'][rmax]['Data']
    after = s['PairedAfter'][rmax]['Data']
    t = s['Type']
    pair_distance[t] = pair_distance[t].append(pd.Series(np.sqrt((before['x'] - after['x'])**2 +
                        (before['y'] - after['y'])**2 +
                        (before['z'] - after['z'])**2)))
    

print('Fraction of learners < 3: {0:.2f}'.
      format(len(pair_distance['learner'][pair_distance['learner'] < 3]) / len(pair_distance['learner'])))
print('Fraction of nonlearners < 3: {0:.2f}'.
      format(len(pair_distance['nonlearner'][pair_distance['nonlearner'] < 3]) / len(pair_distance['nonlearner'])))

data = []
for i in study_types:
     data.append(go.Histogram(x=pair_distance[i], name='{0} Radias {1}'.format(i.capitalize(), rmax)))

hist_layout = go.Layout(
    title='Pair Distance Histogram Radius {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data=data, layout=hist_layout)

py.offline.iplot(hist_fig, filename='pair-distance')

<h1> Histogram of pair distances - past a certain pairing threshold 

In [ ]:
thres_pair=20.0 # Threshold for the minimum value to display on the histogram

hist_data_l_thres = go.Histogram(x=pair_distance['learner'], name='Learners Radius {0}'.format(r),
                          opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1))
hist_data_nl_thres = go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radius {0}'.format(r),
                           opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1))

# Normalized
hist_data_l_N_thres = go.Histogram(x=pair_distance['learner'], name='Learners Radius {0}'.format(r),
                          opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1),histnorm="probability")
hist_data_nl_N_thres = go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radius {0}'.format(r),
                           opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1),histnorm="probability")

hist_layout_Norm = go.Layout(
    title='Pair Distance Histogram Radius Normalized {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig_thres = go.Figure(data=[hist_data_l_thres, hist_data_nl_thres], layout=hist_layout)
hist_fig_Norm_thres = go.Figure(data=[hist_data_l_N_thres, hist_data_nl_N_thres], layout=hist_layout_Norm)

py.offline.iplot(hist_fig_thres, filename='pair-distance_thresholded')
py.offline.iplot(hist_fig_Norm_thres, filename='pair-distance_Norm_thresholded')

<h1> Histogram of intensity ratios

In [ ]:
r  = min(studylist[0]['UnpairedBefore'])

core_ratio = { k : pd.Series() for k in study_types}
core = { k : pd.Series() for k in study_types}

for s in studylist:
    before = s['PairedBefore'][r]['Data']
    after = s['PairedAfter'][r]['Data']
    t = s['Type']
    core_ratio[t] = core_ratio[t].append(pd.Series(before['core'] / after['core']))
    core[t] = core_ratio[t].append(pd.Series(before['core']))
    core[t] = core_ratio[t].append(pd.Series(after['core']))

data = []
for i in study_types:
     data.append(go.Histogram(x=core_ratio[i], name='{0} Radias {1}'.format(i.capitalize(), r)))
    
hist_layout = go.Layout(
    title='Pair Ratio Histogram Radius {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data=data, layout=hist_layout)

py.offline.iplot(hist_fig, filename='core-intensity')

<H2> Basic statistics for a study set

In [ ]:
c1,c2,c3,c4, c5 = [], [], [], [], []
synapses = sp.aggregate_studies(studylist)[0]

for k,v in synapse_utils.group_studies(studylist, group='Type').items():
    c1.append(k)
    c2.append(len(v))
    c3.append(synapses[k]['UnpairedBefore'].shape[0])
    c4.append(synapses[k]['UnpairedAfter'].shape[0])
    c5.append(synapses[k]['PairedBefore'].shape[0])
table = go.Table(
    header=dict(values=['Type', 'Count', 'Before Unpaired', 'After Unpaired', 'Paired'],
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 4),
    cells=dict(values=[c1, c2, c3, c4, c5],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

layout = go.Layout(
    title='Summary Statistics Study Set SYNAPSE:{0}'.format(studyid),
    )

fig = go.Figure(data=[table], layout=layout)

py.offline.iplot(fig, filename = 'statistics_table')

<h2> Dump out CSV Files....

In [ ]:
testDict = {cols[0]:c1,cols[1]:c2,cols[2]:c3,cols[3]:c4,cols[4]:c5,cols[5]:c6}
with open('SynapseDataTable.csv', 'w') as f:  # Just use 'w' mode in 3.x
   w = csv.DictWriter(f, cols)
   w.writeheader()
   w = csv.writer(f,cols,delimiter=',', quoting=csv.QUOTE_NONE)
   for i in range(0,len(c1)):
       w.writerow((testDict[cols[0]][i],testDict[cols[1]][i],
                   testDict[cols[2]][i],testDict[cols[3]][i],
                   testDict[cols[4]][i],testDict[cols[5]][i]))
       
# For every nth row (so that you only look at one radius element at a time)
RowForRad = 3 #look at every RowForRad-th row
filename = 'SynapseDataTable_'+str(RowForRad)+'Rows.csv'
with open(filename, 'w') as f:  # Just use 'w' mode in 3.x
   w = csv.DictWriter(f, cols)
   w.writeheader()
   w = csv.writer(f,cols,delimiter=',', quoting=csv.QUOTE_NONE)
   for i in range(0,len(c1),RowForRad):
       w.writerow((testDict[cols[0]][i],testDict[cols[1]][i],
                   testDict[cols[2]][i],testDict[cols[3]][i],
                   testDict[cols[4]][i],testDict[cols[5]][i]))

<H1> Plot sets of synapses...

In [ ]:
# Get all radii for a specific study

#s_list = [s for s in paired_studies if s['Study'] in s_names]
# Pick which traces you would like displayed

tracelist =['AlignedUnpairedBefore','AlignedUnpairedAfter',
            'AlignedPairedBefore',
            'AlignmentPts']

print('Plotting {0} studies'.format(len(studylist)))

for k,v in synapse_utils.group_studies(studylist, group='Type').items():
    print('{0} {1}'.format(len(v), k))

#Pick the slider mode...
plotmode = 'Study'
#plotmode = 'Trace'
plotmode = 'Studyset'


fig = plot_synapses(studylist, tracelist, plotmode, centroid=False, skipall=True) 

plot_file_name = 'synapse-plot'
py.offline.iplot(fig, filename=plot_file_name)
    
#py.offline.plot(fig,  filename =plot_file_name, output_type='file')
    

<h2> Plot summary of centroids

In [ ]:
centroidlist = []
tracelist =['AlignedUnpairedBefore','AlignedUnpairedAfter','AlignedPairedBefore', 'AlignmentPts']

# Use the smallest available radius as the default if one is not provided.
radius =  min(studylist[0][tracelist[0]])

for t in tracelist: 
    if t != 'AlignmentPts':
        centroidlist.append(t + 'Centroid')
before_traces, after_traces, paired_traces = [], [], []

cnt_before = cnt_after = cnt_paired = 0
before_x = before_y = before_z = 0
after_x = after_y = after_z = 0
paired_x = paired_y = paired_z = 0
maxx,maxy,maxz,minx,miny,minz = -float('inf'),-float('inf'),-float('inf'),float('inf'),float('inf'),float('inf')
    
for t in centroidlist:
    x,y,z, text = [],[],[],[]
    for s in studylist:
        x.append(s[t][radius]['Data']['x'][0])
        y.append(s[t][radius]['Data']['y'][0])
        z.append(s[t][radius]['Data']['z'][0])
        text.append(s['Study'])
        
    maxx = max(x + [maxx])
    maxy = max(y + [maxy])
    maxz = max(z + [maxz])
    minx = min(x + [minx])
    miny = min(y + [miny])
    minz = min(z + [minz])
    
    centroid_trace = go.Scatter3d(
        x = x, y = y, z = z,
        mode='markers',
        name = '{0}'.format(t),
        text = text,
        marker=dict(
            size = 6, color = trace_color(t),
            line=dict(color='black', width=0.8))
        )

    if 'UnpairedBefore' in t:
        before_x = sum(x) + before_x
        before_y = sum(y) + before_y
        before_z = sum(z) + before_z
        cnt_before = cnt_before + len(x)
        before_traces.append(centroid_trace)
    elif 'UnpairedAfter' in t:
        after_x = sum(x) + after_x
        after_y = sum(y) + after_y
        after_z = sum(z) + after_z
        cnt_after = cnt_after + len(x)
        after_traces.append(centroid_trace)
    elif 'PairedBefore' in t:
        paired_x = sum(x) + paired_x
        paired_y = sum(y) + paired_y
        paired_z = sum(z) + paired_z
        cnt_paired = cnt_paired + len(x)
        paired_traces.append(centroid_trace)
        
summary_trace = go.Scatter3d(
    x = [ before_x / cnt_before ], y = [ before_y / cnt_before ], z = [ before_z / cnt_before ],
    mode = 'markers',
    name = 'Centroids Before',
    marker = dict(
        size = 8,
        symbol = 'square',
        color = ['orange']
     )
)
before_traces.append(summary_trace)

summary_trace = go.Scatter3d(
    x = [ after_x / cnt_after ],  y = [ after_y / cnt_after ], z = [ after_z / cnt_after ],
    mode = 'markers',
    name = 'Centroids After',
    marker = dict(
        size = 8,
        symbol = 'square',
        color = ['blue']
     )
)
after_traces.append(summary_trace)

summary_trace = go.Scatter3d(
    x = [ paired_x / cnt_paired ],  y = [ paired_y / cnt_paired ], z = [ paired_z / cnt_paired ],
    mode = 'markers',
    name = 'Centroids Paired',
    marker = dict(
        size = 8,
        symbol = 'square',
        color = ['green']
     )
)
paired_traces.append(summary_trace)

masks = {'study' : 
         { 'centroid' : 
          {
    'all' :     [True] * (len(before_traces) + len(after_traces) + len(paired_traces)),
    'before' :  [True] * len(before_traces) + [False] * (len(after_traces) + len(paired_traces)),
    'after' : [False] * len(before_traces) + [True] * len(after_traces) + [False] * len(paired_traces),
    'paired' : [False] * (len(before_traces) + len(after_traces)) + [True] * len(paired_traces),
          }
         }
    }

layout = position_layout(minx, miny, minz, maxx, maxy, maxz)
layout['title'] = 'Centroids'
layout['updatemenus'] = step_buttons('study', masks, study_types, step='centroid')

fig = dict(data=before_traces + after_traces + paired_traces, layout=layout)

plot_file_name = 'centroid-plot'

py.offline.iplot(fig, filename=plot_file_name)
    
#py.offline.plot(fig,  filename =plot_file_name, output_type='file')

<h1> Plot a contour for a set of studies

In [ ]:
def plot_contours(unpairedbefore,unpairedafter,paired,c0,c1, syn_max, syn_min, title):
    
    
    scl_b = cl.scales['9']['seq']['Blues']
    colorscale_b = [ [ float(i)/float(len(scl_b)-1), scl_b[i] ] for i in range(len(scl_b)) ]
    scl_o = cl.scales['9']['seq']['Oranges']
    colorscale_o = [ [ float(i)/float(len(scl_o)-1), scl_o[i] ] for i in range(len(scl_o)) ]
    scl_g = cl.scales['9']['seq']['Greens']
    colorscale_g = [ [ float(i)/float(len(scl_g)-1), scl_g[i] ] for i in range(len(scl_g)) ]

    for i in range(7):
        colorscale_b[i] = [colorscale_b[i][0], 'white']
        colorscale_o[i] = [colorscale_o[i][0], 'white']
        colorscale_g[i] = [colorscale_o[i][0], 'white']

    data = []
    studyset = {}
    masks = {'studyset': studyset}

    # We have a trace for all/before/all and center of mass for each study type, 
    trace_cnt = (len(study_types)) * 6
    for i in study_types:
        studyset[i] = {j: [False] * trace_cnt for j in ['all', 'before', 'after', 'paired']}

    data = []
    for i in study_types:
        # Set mask for this trace 

        studyset[i]['all'][len(data)] = True
        studyset[i]['before'][len(data)] = True

        data.append(go.Histogram2dcontour(
            histnorm=histnorm,
            contours=dict(showlines=False),
            visible=False,
            x = unpairedbefore[i][coord0], y = unpairedbefore[i][coord1],
            name='before density', 
            colorscale=colorscale_o, reversescale=False, showscale=False, opacity=0.5)
        )

        cm0 = unpairedbefore[i][coord0].sum()/unpairedbefore[i].shape[0]
        cm1 = unpairedbefore[i][coord1].sum()/unpairedbefore[i].shape[0]  
        studyset[i]['all'][len(data)] = True
        studyset[i]['before'][len(data)] = True
        data.append(go.Scatter(
            visible=False,
            x=[cm0], y = [cm1],
            mode = 'markers', marker = dict(size = 10, color = 'orange', symbol = 'cross'))
        )


        studyset[i]['all'][len(data)] = True
        studyset[i]['after'][len(data)] = True
        data.append(go.Histogram2dcontour(
            histnorm=histnorm,
                visible=False,
                x = unpairedafter[i][coord0], y = unpairedafter[i][coord1],
                name='after density', #ncontours=5,
                contours=dict(showlines=False),
             #   xaxis='after_x', yaxis='after_y',
                            colorscale=colorscale_b, reversescale=False, showscale=False, opacity=0.5) 
        )

        cm0 = unpairedafter[i][coord0].sum()/unpairedafter[i].shape[0]
        cm1 = unpairedafter[i][coord1].sum()/unpairedafter[i].shape[0]
        studyset[i]['all'][len(data)] = True
        studyset[i]['after'][len(data)] = True
        data.append(go.Scatter(
            visible=False,
            x=[cm0], y = [cm1],
            mode = 'markers', marker = dict(size = 10, color = 'blue', symbol = 'cross'))
        )

        studyset[i]['all'][len(data)] = True
        studyset[i]['paired'][len(data)] = True
        data.append(go.Histogram2dcontour(
            histnorm=histnorm,
                visible=False,
                contours= dict(showlines=False),
                x = paired[i][coord0], y = paired[i][coord1],
                name='paired density', #ncontours=5,
             #   xaxis='after_x', yaxis='after_y',
                            colorscale=colorscale_g, reversescale=False, showscale=False, opacity=0.5)
        )

        cm0 = paired[i][coord0].sum()/paired[i].shape[0]
        cm1 = paired[i][coord1].sum() /paired[i].shape[0]
        studyset[i]['all'][len(data)] = True
        studyset[i]['paired'][len(data)] = True
        data.append(go.Scatter(
            visible=False,
            x=[cm0], y = [cm1],
            mode = 'markers', marker = dict(size = 10, color = 'green', symbol = 'cross'))
        )


    layout = go.Layout(
        showlegend=False,
        autosize=False,
        title=title,
      #  width=600,
        height=800,
        xaxis=dict(
            domain=[0, 1],
            range=[syn_min[0],syn_max[0]],
            title='X',
            showgrid=False,
            zeroline=False
        ),
        yaxis=dict(
            domain=[0, .7],
            title='Z',
            range=[syn_min[2],syn_max[2]],
            scaleanchor='x',
            showgrid=False,
            zeroline=False
        ),
        hovermode='closest',
    )

    for i,v in enumerate(masks['studyset']['learner']['all']):
        data[i]['visible'] = v
    layout['updatemenus'] = step_buttons('studyset', masks, study_types, showlegend=False, 
                                         title=title,
                                         skipall=True)
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig, filename=plot_file_name)

In [ ]:
# Contour plots.....

# The number of slices in Y you want.
#nbins = 5
nbins = 1

plot_file_name = 'centroid-plot'
coord0 = 'x'
coord1 = 'z'
histnorm='percent'

synapses, smax, smin = sp.aggregate_studies(studylist)

c = 'y'
cmax = smax[1]
cmin = smin[1]
binsize = (cmax - cmin) / nbins
for i in study_types:
    synapses[i]['UnpairedBefore']['Slice'] = pd.cut(synapses[i]['UnpairedBefore'][c],
                  [cmin + i * binsize for i in range(nbins + 1)],
                  labels=list(range(nbins)), include_lowest=True)
    synapses[i]['UnpairedAfter']['Slice']  = pd.cut(synapses[i]['UnpairedAfter'][c],
                  [cmin + i * binsize for i in range(nbins + 1)],
                  labels=list(range(nbins)), include_lowest=True)     
    synapses[i]['PairedBefore']['Slice'] = pd.cut(synapses[i]['PairedBefore'][c],
                  [cmin + i * binsize for i in range(nbins + 1)],
                  labels=list(range(nbins)), include_lowest=True)
    
for slice in range(nbins):
    unpaired_before, unpaired_after, paired = {},{},{}
    for i in study_types:
        unpaired_before[i] = synapses[i]['UnpairedBefore'][synapses[i]['UnpairedBefore']['Slice'] == slice]
        unpaired_after[i] = synapses[i]['UnpairedAfter'][synapses[i]['UnpairedAfter']['Slice'] == slice]
        paired[i] = synapses[i]['PairedBefore'][synapses[i]['PairedBefore']['Slice'] == slice]
    plot_contours(unpaired_before, unpaired_after, paired, coord0, coord1, smax, smin,
                  'Synapse Density Y slice {0}/{1} '.format(slice,nbins))


<h1> Histogram of Densities

In [ ]:
# Look at density distributions

binned_synapses = sp.bin_synapses(studylist, nbins = 10)

for t in list(binned_synapses):
    data = []
    for i in ['UnpairedBefore', 'UnpairedAfter', 'PairedBefore']:
        pts = binned_synapses[t][i].to_dataframe()
        cnt = float(pts.sum())
        pts = pts.loc[pts[i] != 0][i] / cnt
        data.append(go.Histogram(x=pts, name='{0} Density {1}'.format(t,i)))
    
    hist_layout = go.Layout(
        title='Synaptic Density {0}'.format(t),
        xaxis=dict(title='Density'),
        yaxis=dict(title='Count'),
        bargap=0.2, bargroupgap=0.1
    )
    hist_fig = go.Figure(data=data, layout=hist_layout)
    py.offline.iplot(hist_fig, filename='core-intensity')



<H1> Thresholded Density Center of Mass

In [ ]:
threshold = .8
nbins = 10

binned_synapses = sp.bin_synapses(studylist, nbins = nbins)
d = sp.synapse_density(binned_synapses, axis='y', mode='density', threshold=threshold)

c1,c2,c3,c4,c5,c6,c7 = [], [], [], [], [], [], []

for i in study_types:
    c1.append(i)
    c2.append(d[i]['UnpairedBefore'].attrs['center_of_mass'][0])
    c3.append(d[i]['UnpairedBefore'].attrs['center_of_mass'][1])
    c4.append(d[i]['UnpairedAfter'].attrs['center_of_mass'][0])
    c5.append(d[i]['UnpairedAfter'].attrs['center_of_mass'][1])
    c6.append(d[i]['PairedBefore'].attrs['center_of_mass'][0])
    c7.append(d[i]['PairedBefore'].attrs['center_of_mass'][1])
table = go.Table(
     columnwidth = [60] + [30],
     header=dict(values=['Type','Before X', 'Before Z','After X', 'After Z', 'Paired X','Paired Z'],
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 4),
    cells=dict(values=[c1, c2, c3, c4, c5, c6, c7],
               format =  [None,'.2f'],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

layout = go.Layout(
    title='Centroid Location Threshold:{0}'.format(threshold),
    )

fig = go.Figure(data=[table], layout=layout)

py.offline.iplot(fig, filename = 'statistics_table')

In [ ]:
plot_file_name = 'synapse-headmap'


def plot_synapse_heatmap(synapse_density):
    
    scl_b = cl.scales['9']['seq']['Blues']
    colorscale_b = [ [ float(i)/float(len(scl_b)-1), scl_b[i] ] for i in range(len(scl_b)) ]
    scl_o = cl.scales['9']['seq']['Oranges']
    colorscale_o = [ [ float(i)/float(len(scl_o)-1), scl_o[i] ] for i in range(len(scl_o)) ]
    scl_g = cl.scales['9']['seq']['Greens']
    colorscale_g = [ [ float(i)/float(len(scl_g)-1), scl_g[i] ] for i in range(len(scl_g)) ]

    colorscale_b[0] = [colorscale_b[0][0], 'white']
    colorscale_o[0] = [colorscale_o[0][0], 'white']
    colorscale_g[0] = [colorscale_o[0][0], 'white']

    data = []
    studyset = {}
    masks = {'studyset': studyset}

    # We have a trace for all/before/all and center of mass for each study type, 
    trace_cnt = (len(study_types)) * 6
    for i in study_types:
        studyset[i] = {j: [False] * trace_cnt for j in ['all', 'before', 'after', 'paired']}
    
    for i in study_types:
        studyset[i]['all'][len(data)] = True
        studyset[i]['before'][len(data)] = True

        density = synapse_density[i]['UnpairedBefore']
        data.append(go.Heatmap(
            visible=False,
            z = density.data, x = density.coords['x'].data, y = density.coords['z'].data,
            name='before density', 
            colorscale=colorscale_o, reversescale=False, showscale=False, opacity=0.5)
        )

        studyset[i]['all'][len(data)] = True
        studyset[i]['before'][len(data)] = True
        center_of_mass = density.attrs['center_of_mass']  
        print('{0} before {1:.2f} {2:.2f}'.format(i,center_of_mass[0],center_of_mass[1]))
        data.append(go.Scatter(
            visible=False,
            x=[center_of_mass[0]], y = [center_of_mass[1]],
            mode = 'markers', marker = dict(size = 10, color = 'orange', symbol = 'cross'))
        )

        studyset[i]['all'][len(data)] = True
        studyset[i]['after'][len(data)] = True
        density = synapse_density[i]['UnpairedAfter']
        data.append(go.Heatmap(
                visible=False,
                z = density.data, x = density.coords['x'].data, y = density.coords['z'].data,
                name='after density', #ncontours=5,
             #   xaxis='after_x', yaxis='after_y',
                            colorscale=colorscale_b, reversescale=False, showscale=False, opacity=0.5)
            )

        studyset[i]['all'][len(data)] = True
        studyset[i]['after'][len(data)] = True
        center_of_mass = density.attrs['center_of_mass'] 
        print('{0} after {1:.2f} {2:.2f}'.format(i,center_of_mass[0],center_of_mass[1]))
        data.append(go.Scatter(
            visible=False,
            x=[center_of_mass[0]], y = [center_of_mass[1]],
            mode = 'markers', marker = dict(size = 10, color = 'blue', symbol = 'cross'))
        )

        studyset[i]['all'][len(data)] = True
        studyset[i]['paired'][len(data)] = True
        density = synapse_density[i]['PairedBefore']
        data.append(go.Heatmap(
            visible=False,
                z = density.data, x = density.coords['x'].data, y = density.coords['z'].data,
                name='paired density',
            colorscale=colorscale_g, reversescale=False, showscale=False, opacity=0.5)
                   )

        studyset[i]['all'][len(data)] = True
        studyset[i]['paired'][len(data)] = True
        center_of_mass = density.attrs['center_of_mass']
        print('{0} paired {1:.2f} {2:.2f}'.format(i,center_of_mass[0],center_of_mass[1]))
        data.append(go.Scatter(
            visible=False,
            x=[center_of_mass[0]], y = [center_of_mass[1]],
            mode = 'markers', marker = dict(size = 10, color = 'green', symbol = 'cross'))
        )

    smin = synapse_density['learner'].attrs['min']
    smax = synapse_density['learner'].attrs['max']
    layout = go.Layout(
        showlegend=False,
        autosize=False,
        title='Synapse Density: Learner All',
      #  width=600,
        height=700,
        xaxis=dict(
            domain=[0, 1],
            range=[smin[0],smax[0]],
            title='X',
            showgrid=False,
            zeroline=False
        ),
        yaxis=dict(
            domain=[0, .7],
            title='Z',
            range=[smin[1],smax[1]],
            scaleanchor='x',
            showgrid=False,
            zeroline=False
        ),
        hovermode='closest',
    )

    for i,v in enumerate(masks['studyset']['learner']['all']):
        data[i]['visible'] = v
    layout['updatemenus'] = step_buttons('studyset', masks, study_types, showlegend=False, title='Synapse Density: ',
                                         skipall=True)
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig, filename=plot_file_name)

In [ ]:
threshold = .8
nbins = 10

binned_synapses = sp.bin_synapses(studylist, nbins = nbins)
d = sp.synapse_density(binned_synapses, axis='y', mode='density', threshold=threshold)

plot_synapse_heatmap(d)

<H1> Plot Vector difference between unpaired before and after and paired.

In [ ]:
# 2D version.....

threshold = .8
nbins = 10
binned_synapses = sp.bin_synapses(studylist, nbins = nbins)
synapse_density = sp.synapse_density(binned_synapses, axis='y', mode='density', threshold=threshold)

data = []
data_before = []
data_after = []
data_diff = []
studyset = {}
masks = {'studyset': studyset}

study_colors = dict(zip(list(study_types), ['red', 'nonlearner''green', 'blue', 'orange', 'purple' ]))

# We have a trace for all/before/after  for each study type, 
trace_cnt = (len(study_types)) * 2

for i in study_types + ['all']:
    studyset[i] = {j: [False] * trace_cnt for j in ['all', 'before', 'after']}
# Look at density distributions

maxx,maxy,maxz,minx,miny,minz = -float('inf'),-float('inf'),-float('inf'),float('inf'),float('inf'),float('inf')
for i in study_types:
    before_com = synapse_density[i]['UnpairedBefore'].attrs['center_of_mass']
    after_com = synapse_density[i]['UnpairedAfter'].attrs['center_of_mass']
    paired_com = synapse_density[i]['PairedBefore'].attrs['center_of_mass']
    
    
    studyset[i]['all'][len(data)] = True
    studyset[i]['before'][len(data)] = True
    x, y  = [ before_com[i] - paired_com[i] for i in range(len(paired_com))]
    scale = (x**2 * y**2)**.5
    maxx = x if x > maxx else maxx
    maxy = y if y > maxy else maxy
    minx = x if x < minx else minx
    miny = y if y < miny else miny
    
    print('vector before {0}: {1}, {2}'.format(i, x, y))
    data_before.append(go.Scatter(
                    name='{0}  {1}'.format(i,'before'), 
                    mode = 'lines', visible = True,
                    x=[0,x], y = [0,y], 
                    line = dict(color=study_colors[i])
                    )
               )

    x, y  = [ after_com[i] - paired_com[i] for i in range(len(after_com))]
    maxx = x if x > maxx else maxx
    maxy = y if y > maxy else maxy
    minx = x if x < minx else minx
    miny = y if y < miny else miny
    
    print('vector after {0}: {1}, {2}'.format(i, x,y))
    data_after.append(go.Scatter(name='{0} {1}'.format(i,'after'), mode = 'lines',
            x=[0,x], y = [0,y], visible=True,
            line = dict(color=study_colors[i])
                            )
               )
    
    x, y  = [ after_com[i] - before_com[i] for i in range(len(after_com))]
    data_diff.append(go.Scatter(name='{0} {1}'.format(i,'after'), mode = 'lines',
            x=[0,x], y = [0,y], visible=True,
            line = dict(color=study_colors[i])
                            )
               )
   
layout = go.Layout(height = 700,
        title='Density Vector Distance',
        xaxis=dict(
                range=[minx,maxx],
                title='X'
            ),
        yaxis=dict(
                title='Z',
                range=[miny,maxy],
            )
    )

layout['title'] = 'Density Vector Distance: Before'
fig = go.Figure(data=data_before, layout=layout)
py.offline.iplot(fig, filename='core-intensity')

layout['title'] = 'Density Vector Distance: After'
fig = go.Figure(data=data_after, layout=layout)
py.offline.iplot(fig, filename='core-intensity')

layout['title'] = 'Density Vector Distance: Diff'
fig = go.Figure(data=data_diff, layout=layout)
py.offline.iplot(fig, filename='core-intensity')




In [ ]:
threshold = .8
nbins = 10
binned_synapses = sp.bin_synapses(studylist, nbins = nbins)
synapse_density = sp.synapse_density(binned_synapses, axis='y', mode='density', threshold=threshold)
#synapse_density = sp.synapse_density3d(binned_synapses, threshold=threshold)

data = []
studyset = {}
masks = {'studyset': studyset}

# We have a trace for all/before/after  for each study type, 
trace_cnt = (len(study_types)) * 2

for i in study_types + ['all']:
    studyset[i] = {j: [False] * trace_cnt for j in ['all', 'before', 'after']}
# Look at density distributions

maxx,maxy,maxz,minx,miny,minz = -float('inf'),-float('inf'),-float('inf'),float('inf'),float('inf'),float('inf')
for i in study_types:
    before_com = synapse_density[i]['UnpairedBefore'].attrs['center_of_mass']
    after_com = synapse_density[i]['UnpairedAfter'].attrs['center_of_mass']
    paired_com = synapse_density[i]['PairedBefore'].attrs['center_of_mass']
    
    studyset['all']['all'][len(data)] = True
    studyset['all']['before'][len(data)] = True
    
    studyset[i]['all'][len(data)] = True
    studyset[i]['before'][len(data)] = True
    x, y, z  = [ before_com[i] - paired_com[i] for i in range(len(paired_com))]
    maxx = x if x > maxx else maxx
    maxy = y if y > maxy else maxy
    maxz = z if z > maxz else maxz
    minx = x if x < minx else minx
    miny = y if y < miny else miny
    minz = z if z < minz else minz
    
    print('vector before {0}: {1}, {2}, {3}'.format(i, x,y,z))
    data.append(go.Scatter(
                    name='{0}  {1}'.format(i,'before'), 
                    mode = 'lines', visible = False,
                    x=[0,x], y = [0,y], 
#        z = [0,z],
                    line = dict(color='orange')
                    )
               )

    studyset[i]['all'][len(data)] = True
    studyset[i]['after'][len(data)] = True
    studyset['all']['after'][len(data)] = True
    studyset['all']['all'][len(data)] = True
    x, y, z  = [ after_com[i] - paired_com[i] for i in range(len(after_com))]
    maxx = x if x > maxx else maxx
    maxy = y if y > maxy else maxy
    maxz = z if z > maxz else maxz
    minx = x if x < minx else minx
    miny = y if y < miny else miny
    minz = z if z < minz else minz
    
    print('vector after {0}: {1}, {2}, {3}'.format(i, x,y,z))

    data.append(go.Scatter3d(name='{0} {1}'.format(i,'after'), mode = 'lines',
            x=[0,x], y = [0,y], z = [0,z], visible=False,
            line = dict(color='blue')
                            )
               )
    
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig, filename='core-intensity')
    
layout = go.Layout(height = 700,
        title='Density Vector Distance',
        scene= dict(
            xaxis=dict(
                range=[minx,maxx],
                title='X'
            ),
            yaxis=dict(
                title='Z',
                range=[miny,maxy],
            ),
            zaxis=dict(
                title='Z',
                range=[minz,maxz],
            )
        ),
    )

for i,v in enumerate(masks['studyset']['all']['all']):
        data[i]['visible'] = v
layout['updatemenus'] = step_buttons('studyset', masks, study_types, showlegend=False, 
                                     title='Density Vector Distance: ',
                                         skipall=False)
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig, filename='core-intensity')



In [ ]:
def density_fig(title,bx,by,bz,ax,ay,az,
                centroid_before, centroid_after, bcolors, acolors, bcmap,acmap):
    max_x = max([max(ax),max(bx)])
    min_x = min([min(ax),min(bx)])
    max_y = max([max(ay),max(by)])
    min_y = min([min(ay),min(by)])
    max_z = max([max(az),max(bz)])
    min_z = min([min(az),min(bz)])

    data = [  
        go.Scatter3d(
            visible=True,
            x = [v['x'] for i,v in centroid_before.items()],
            y = [v['y'] for i,v in centroid_before.items()],
            z = [v['z'] for i,v in centroid_before.items()],
            hovertext=['Before Centroid {0}'.format(i) for i,v in centroid_before.items()],
            text= ['Before Centroid'],
            mode='markers', name='Before Centroid',
            marker=dict(
                symbol='cross',               
                size=8, color='blue',
                line=dict(color='blue', width=1))
        ),
        
        go.Scatter3d(
            visible=True,
            x=bx[bcolors == -1], y=by[bcolors == -1] , z=bz[bcolors == -1],
            hovertext= 'Before Cluster Noise',
            mode='markers', name='Before Noise',
            marker=dict(
                symbol='circle',  
                size=3, color='lightblue', opacity=0.2,
            )
        ),
                                    
        go.Scatter3d(
            visible=True,
            x=bx[bcolors > -1], y=by[bcolors > -1] , z=bz[bcolors > -1],
            hovertext= ['Before Cluster {0}'.format(i) for i in bcolors],
            mode='markers', name='Before',
            marker=dict(
                symbol='circle',  
                colorscale=bcmap,
                size=4, color='blue', opacity=0.9,
            )
        ),
        
        go.Scatter3d(
            visible=True,
            x = [v['x'] for i,v in centroid_after.items()],
            y = [v['y'] for i,v in centroid_after.items()],
            z = [v['z'] for i,v in centroid_after.items()],
            hovertext=['After Centroid {0}'.format(i) for i,v in centroid_after.items()],
            text= ['After Centroid'],
            mode='markers', name='After Centroid',
            marker=dict(
                symbol='cross',               
                size=8, color='red',
                line=dict(color='red', width=1))
        ),
        
        go.Scatter3d(
            visible=True,
            x=ax[acolors == -1], y=ay[acolors == -1] , z=az[acolors == -1],
            hovertext= 'After Cluster Noise',
            mode='markers', name='After Noise',
            marker=dict(
                symbol='circle',  
                size=3, color='lightred', opacity=0.2,
            )
        ),
        go.Scatter3d(
            visible=True,
            x=ax[acolors > -1], y=ay[acolors > -1], z=az[acolors > -1],
            hovertext=['After Cluster {0}'.format(i) for i in acolors],
            mode='markers', name='After',
            marker=dict(
                symbol='circle',
                size=4, color='red', opacity=0.9,
 #               line=dict(color='red', width=0.8)
            )
        )
        

          
    ]

    layout = go.Layout(
        height=900,
        showlegend=True,
        title = title,
        scene=dict(
            xaxis=dict(title='X Axis',
                       color='red',
                       range=[min_x,max_x]),
            yaxis=dict(title='Y Axis',
                       color='green',
                       range=[min_y,max_y]),
        zaxis=dict(title='Z Axis',
                   color='blue', 
                   range=[min_z,max_z]),
        aspectmode='cube',
        domain=dict(y=[0,.99]),
        annotations=plot_config.position_annotations(min_x,min_y,min_z,max_x,max_y,max_z)
        #    dragmode='turntable',
    )
)

    fig = go.Figure(data=data, layout=layout)
    return fig

In [ ]:
synapses, syn_max, syn_min = sp.aggregate_studies(studylist)

#learners = synapse_utils.group_studies(studylist)['learner']
#nonlearners = synapse_utils.group_studies(studylist)['nonlearner']

t = ['learner','nonlearner']
# Parameters that control clustering.  The pairs are for before/after.

csize_fraction = {'learner':(.02,.02), 'nonlearner':(.02,.02)}
msamples_fraction = {'learner': (.2,.9), 'nonlearner': (.3,.6)}
single_cluster=True

before_synapses, after_synapses = {},{}
before_clusterer, after_clusterer = {}, {}
centroid_before, centroid_after = {},{}

for i in t:
    before_synapses[i] = synapses[i]['UnpairedBefore']
    after_synapses [i]= synapses[i]['UnpairedAfter']

    min_cluster_size = int(csize_fraction[i][0]* len(before_synapses[i]))
    min_samples = int(min_cluster_size*msamples_fraction[i][0])
    before_clusterer[i] = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, 
                                   allow_single_cluster=single_cluster)
    before_clusterer[i].fit(before_synapses[i][['x','y','z']].values)

    min_cluster_size = int(csize_fraction[i][1]* len(after_synapses[i]))
    min_samples = int(min_cluster_size*msamples_fraction[i][1])
    after_clusterer[i] = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, 
                                  allow_single_cluster=single_cluster)
    after_clusterer[i].fit(after_synapses[i][['x','y','z']].values)

    # Compute centroid....
    centroid_before[i], centroid_after[i] = {},{}
    for c in range(-1, before_clusterer[i].labels_.max()+1):
        s = before_synapses[i][before_clusterer[i].labels_ == c]
        centroid_before[i][c] = s.sum() / len(s)
    for c in range(-1, after_clusterer[i].labels_.max()+1):
        s = after_synapses[i][after_clusterer[i].labels_ == c]
        centroid_after[i][c] = s.sum() / len(s)

    clusterer = before_clusterer[i]
    print('{} before pts: {} cluster_size:{} min_samples:{}'.format(i,len(clusterer.labels_), 
                                                             clusterer.min_cluster_size, 
                                                             clusterer.min_samples))
    print('  Number of before clusters: {0}'.format(before_clusterer[i].labels_.max()+1))

    for c in range(-1,before_clusterer[i].labels_.max()+1):
        cluster = before_synapses[i][before_clusterer[i].labels_ == c][['x','y','z']]
        hull = ConvexHull(cluster)
        print('    Cluster {0}: points {1}  area {2:.2f} density: {3:.2f} centroid: ({4:.2f},{5:.2f},{6:.2f}) '.\
              format(c,len(cluster),hull.area, hull.area/len(cluster),
                centroid_before[i][c]['x'],centroid_before[i][c]['y'],centroid_before[i][c]['z']))
        
#    print(after_clusterer[i])
    clusterer = after_clusterer[i]
    print('{} after pts: {} cluster_size:{} min_samples:{}'.format(i,len(clusterer.labels_), 
                                                             clusterer.min_cluster_size, 
                                                             clusterer.min_samples))
    print('  Number of after clusters: {0}'.format(after_clusterer[i].labels_.max()+1))
    for c in range(-1,after_clusterer[i].labels_.max()+1):
        cluster = after_synapses[i][after_clusterer[i].labels_ == c][['x','y','z']]
        hull = ConvexHull(cluster)
        print('    Cluster {0}: points {1}  area {2:.2f} density: {3:.2f} centroid: ({4:.2f},{5:.2f},{6:.2f}) '.\
              format(c,len(cluster),hull.area, hull.area/len(cluster),
                centroid_after[i][c]['x'],centroid_after[i][c]['y'],centroid_after[i][c]['z']))
   

In [ ]:
# Specify which clusters to show.
show_clusters = {}
show_clusters['learner'] = {'before':[-1,0],'after':[-1,0]}
show_clusters['nonlearner'] = {'before':[-1,0], 'after': [-1,0]}
                                         
for i in ['learner','nonlearner']:
    cluster_mask = np.isin(before_clusterer[i].labels_,show_clusters[i]['before'])
    bx = before_synapses[i][cluster_mask]['x']
    by = before_synapses[i][cluster_mask]['y']
    bz = before_synapses[i][cluster_mask]['z']
    bcolors = before_clusterer[i].labels_[cluster_mask]

    cluster_mask = np.isin(after_clusterer[i].labels_,show_clusters[i]['after'])
    ax = after_synapses[i][cluster_mask]['x']
    ay = after_synapses[i][cluster_mask]['y']
    az = after_synapses[i][cluster_mask]['z']
    acolors = after_clusterer[i].labels_[cluster_mask]

    # Remap color of a cluster....
    bcmap = 'Blues'
    bcmap = [[0,'gray'],[1,'blue']],
    acmap = [[0,'gray'],[1.0,'red']],
    acmap = 'Reds'

    fig = density_fig('Density {}'.format(i), bx, by, bz, ax,ay,az,
                      centroid_before[i], centroid_after[i], bcolors, acolors, bcmap,acmap)
    py.offline.iplot(fig, filename='core-intensity')

In [ ]:
import seaborn as sns
before_clusterer.condensed_tree_.plot(select_clusters=True)
after_clusterer.condensed_tree_.plot()